<a href="https://colab.research.google.com/github/raulbs7/Machine-Learning-Techniques-Project/blob/master/NLP_Supervised_Project/NLP_Hatred_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CLASSIFICATION PROJECT

In [11]:
from google.colab import files
import pandas as pd
import nltk
import io
import re
import string
from collections import Counter
from nltk.tokenize import sent_tokenize, word_tokenize
try:
    import cPickle as pickle
except ImportError: 
    import pickle

nltk.download("popular")
nltk.download('vader_lexicon') #sentiment analysis
nltk.download('twython') #twitter

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

False

## Preprocessing

To preprocess the data, first it is needed to catch the data from the csv files.

In [2]:
def upload_dataframes (index_fields):
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
    df = pd.read_csv(io.StringIO(uploaded[fn].decode('utf-8')), index_col = index_fields)
    return df

In [3]:
tweets = upload_dataframes([])
tweets.head()

Saving labeled_data.csv to labeled_data.csv
User uploaded file "labeled_data.csv" with length 2546446 bytes


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


One important thing is to remove symbols which are noise and not provide any information to the analysis of the text. The symbols that they are going to be removed are the next ones: ! " $ % & ( ) = _ * ^ ¡ @

In [4]:
def remove_symbols(text):
  return re.sub('[!"\$%\&\(\)=_\*\ˆ¡@]', '', text)

In [5]:
tweets['tweet'] = tweets['tweet'].apply(remove_symbols)
tweets.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,RT mayasolovely: As a woman you shouldn't com...
1,1,3,0,3,0,1,RT mleew17: boy dats cold...tyga dwn bad for ...
2,2,3,0,3,0,1,RT UrKindOfBrand Dawg RT 80sbaby4life: You ev...
3,3,3,0,2,1,1,RT CGAnderson: vivabased she look like a tranny
4,4,6,0,6,0,1,RT ShenikaRoberts: The shit you hear about me...


Next step in preprocessing is to remove the capital letters.

In [6]:
def remove_capital_let(text):
  return text.lower()

In [7]:
tweets['tweet'] = tweets['tweet'].apply(remove_capital_let)
tweets.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,rt mayasolovely: as a woman you shouldn't com...
1,1,3,0,3,0,1,rt mleew17: boy dats cold...tyga dwn bad for ...
2,2,3,0,3,0,1,rt urkindofbrand dawg rt 80sbaby4life: you ev...
3,3,3,0,2,1,1,rt cganderson: vivabased she look like a tranny
4,4,6,0,6,0,1,rt shenikaroberts: the shit you hear about me...


After that, it is necessary to correct wrong words of the text, in order to have a better analysis and to create a text sequence more understandable for our NLP algorithms.

In [8]:
uploaded = files.upload()
file_name = "big.txt"
raw = uploaded[file_name].decode("utf-8")

Saving big.txt to big.txt


In [9]:
def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(raw))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [ ]:
tweets['tweet'] = tweets['tweet'].apply(correction)
tweets.head()

Next, it can be optional to remove contractions to have a better tokenization.

In [ ]:
def remove_contractions(text):
  text = re.sub('isn\'t', 'is not', text)
  text = re.sub('aren\'t', 'are not', text)
  text = re.sub('wasn\'t', 'was not', text)
  text = re.sub('hadn\'t', 'had not', text)
  text = re.sub('haven\'t', 'have not', text)
  text = re.sub('wasn\'t', 'was not', text)
  text = re.sub('weren\'t', 'were not', text)
  text = re.sub('can\'t', 'can not', text)
  text = re.sub('doesn\'t', 'does not', text)
  text = re.sub('couldn\'t', 'could not', text)
  text = re.sub('shouldn\'t', 'should not', text)
  text = re.sub('don\'t', 'do not', text)
  text = re.sub('didn\'t', 'did not', text)
  text = re.sub('mustn\'t', 'must not', text)
  text = re.sub('might\'ve', 'might not', text)
  text = re.sub('won\'t', 'will not', text)
  text = re.sub('wouldn\'t', 'would not', text)
  text = re.sub('what\'s', 'what is', text)
  text = re.sub('where\'s', 'where is', text)
  text = re.sub('who\'s', 'who is', text)
  text = re.sub('who\'ll', 'who will', text)
  text = re.sub('that\'s', 'that is', text)
  text = re.sub('there\'s', 'there is', text)
  text = re.sub('i\'ll', 'i will', text)
  text = re.sub('i\'m', 'i am', text)
  text = re.sub('i\'have', 'i have', text)
  text = re.sub('i\'d', 'i would', text)
  text = re.sub('i\'d', 'i had', text)
  text = re.sub('you\'re', 'you are', text)
  text = re.sub('you\'ve', 'you have', text)
  text = re.sub('you\'ll', 'you will', text)
  text = re.sub('you\'d', 'you would', text)
  text = re.sub('you\'d', 'you had', text)
  text = re.sub('he\'s', 'he is', text)
  text = re.sub('he\'ll', 'he will', text)
  text = re.sub('she\'s', 'she is', text)
  text = re.sub('she\'ll', 'she will', text)
  text = re.sub('it\'s', 'it is', text)
  text = re.sub('it\'ll', 'it will', text)
  text = re.sub('we\'re', 'we are', text)
  text = re.sub('we\'ve', 'we have', text)
  text = re.sub('we\'ll', 'we will', text)
  text = re.sub('we\'d', 'we would', text)
  #text = re.sub('we\'d', 'we had', text)
  text = re.sub('they\'re', 'they are', text)
  text = re.sub('they\'ve', 'they have', text)
  text = re.sub('they\'ll', 'they will', text)
  text = re.sub('they\'d', 'they would', text)
  #text = re.sub('they\'d', 'they had', text)

Due to the fact that tweets are being treated, there is a lot of probability to have some emojis in our data. These emojis are represented in our data as a code. Our goal, is to convert this emojis in tokenizable and meaningful words.

In [ ]:
with open('Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

def convert_emojis_to_word(text):
    for emot in Emoji_Dict:
        text = re.sub(r'('+emot+')', "_".join(Emoji_Dict[emot].replace(",","").replace(":","").split()), text)
    return text

In [ ]:
tweets['tweet'] = tweets['tweet'].apply(correction)
tweets.head()

One of the most important steps is the tokenization of the text by sentences or words.

In [ ]:
sentences = sent_tokenize(tweets['tweet'])
words = word_tokenize(tweets['tweet'])